## librerias

In [1]:
import urllib.request
import requests
import sys                                                  # built in
from bs4 import BeautifulSoup
import pandas as pd
from pandas.io.json import json_normalize
import json                                                 # built in
import ast                                                  # built in


import time

import pandas as pd

from multiprocessing import Pool

import os

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


# set options to be headless, ..

options = Options()
options.add_argument('--blink-settings=imagesEnabled=false')

#options.add_argument('--headless')
#options.add_argument('--no-sandbox')
#options.add_argument('--disable-dev-shm-usage')

# abc


## codigo driver get url


In [31]:
# pagina de busqueda

url_air = 'https://casadeplaya.com/busquedas'



In [32]:
driver = webdriver.Chrome(executable_path=r"C:\chromedriver_win32\chromedriver.exe", options=options)
driver.get(url_air)

In [33]:
# get html
page_detailed = driver.page_source

# close the driver
driver.quit()

# BS
detail_soup = BeautifulSoup(page_detailed)

In [ ]:
# listings = detail_soup.find_all('div', 'card  edit_hover_class')  

listings = detail_soup.find_all('div', 'card-block')    # card-block

#listings

In [12]:
#len(listings)

1

## amenities

In [36]:
a = detail_soup.find_all('div', 'icons flex justify-between mb-2')   

In [37]:
a[0].find_all('span')   #[0].get_text()

[<span>4</span>,
 <span>3</span>,
 <span>12</span>,
 <span>--</span>,
 <span>2</span>]

In [38]:
a[0].find_all('title')

[<title>habitaciones</title>,
 <title>wc</title>,
 <title>cama</title>,
 <title>plazas</title>,
 <title>carro</title>]

In [39]:
# sometimes there are more elements within
[a.get_text() for a in a[0].find_all('span')]

['4', '3', '12', '--', '2']

In [40]:
import json

def extract_amenities(soup):
    amenities = soup.find_all('div', {'class': 'icons flex justify-between mb-2'})
    
    amenities_dict = {}
    for amenity in amenities:
        header = amenity.find_all('title')
        header = [v.find(text=True) for v in header]
        values = amenity.find_all('span')
        values = [v.find(text=True) for v in values]
        
        amenities_dict['amenity_' + str(header)] = values
        
    return json.dumps(amenities_dict)

In [41]:
extract_amenities(detail_soup)

'{"amenity_[\'habitaciones\', \'wc\', \'cama\', \'plazas\', \'carro\']": ["5", "4", "14", "16", "4"]}'

## all features

In [7]:
RULES_SEARCH_PAGE = {
    'url': {'tag': 'a', 'get': 'href', 'order':-1},  
    'name': {'tag': 'h4', 'class': 'truncate'},
    'ubicacion': {'tag': 'p'},
    'venta mensual': {'tag':'em','title':'Venta US$'},
    'alquiler mensual': {'tag': 'em', 'title': 'Alquiler mensual US$','order':-1},
    'likes': {'tag': 'small', 'class': 'id'}
}


# Third Generation :)
def extract_element(listing_html, params):
    # 1. Find the right tag
    if 'class' in params:
        elements_found = listing_html.find_all(params['tag'], params['class'])
    else:
        elements_found = listing_html.find_all(params['tag'])

    # 2. Extract the right element
    tag_order = params.get('order', 0)
    element = elements_found[tag_order]
        
    # 3. Get text
    if 'get' in params:
        output = element.get(params['get'])
    else:
        output = element.get_text()

    return output

In [ ]:
for feature in RULES_SEARCH_PAGE:
    try:
        print(f"{feature}: {extract_element(listings[0], RULES_SEARCH_PAGE[feature])}")
    except:
        print(f"{feature}: empty")

In [66]:
def extract_soup_js(listing_url, waiting_time=3):
    """Extracts HTML from JS pages: open, wait, click, wait, extract"""

    options = Options()
    options.add_argument('--headless')
    options.add_argument('--blink-settings=imagesEnabled=false')
    driver = webdriver.Chrome(executable_path=r"C:\chromedriver_win32\chromedriver.exe", options=options)

    driver.get(listing_url)
    time.sleep(waiting_time)

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
    detail_page = driver.page_source

    driver.quit()

    return BeautifulSoup(detail_page, features='html.parser')

In [54]:
def extract_page_features(soup, rules):
    features_dict = {}
    for feature in rules:
        try:
            features_dict[feature] = extract_element(soup, rules[feature])
        except:
            features_dict[feature] = 'empty'
    
    return features_dict

In [56]:
def build_urls(main_url, pages_total=3):               # # pages_found = 4
    url_list = []
    for i in range(pages_total):
      if i==0:        
        url_pagination = main_url                                                   # ver cómo cambia la pagina
        url_list.append(url_pagination)
      else:
        offset = i+1
        url_pagination = main_url + f'?page={offset}'                               # ver cómo cambia la pagina
        url_list.append(url_pagination)
    
    return url_list

In [65]:
# build a list of URLs
url_list = build_urls(url_air, pages_total=9)
len(url_list)


9

In [67]:
# Scrape pages

def process_detail_page(url_list):
        
    features_list = []

    for i in range(len(url_list)):

        url1 = url_list[i]
        soup = extract_soup_js(url1, waiting_time=3)

        casas = soup.find_all('div', 'card-block')      # luego cambiar para varias casas

        for listing in casas:
            features = extract_page_features(listing, RULES_SEARCH_PAGE)
            features['amenities'] = extract_amenities(listing)
            features_list.append(features)

    return features_list


## pruebas

In [ ]:
# try for one page
#base_features = process_detail_page(url_list[:1])

In [49]:
#base_features

NameError: name 'base_features' is not defined

In [19]:
#soup = extract_soup_js(url_air, waiting_time=[3, 1])

#ds = soup.find_all('div', 'card-block')
#ds[3]


# final

In [68]:
# t0 = time.time()
detail_features = process_detail_page(url_list)
# print(time.time() - t0)

In [69]:
print(len(detail_features),"\n",detail_features[7])

193 
 {'url': '/inmuebles/6849-casa-en-pulpos', 'name': 'Casa en Pulpos', 'ubicacion': 'Km 41 Pan. Sur Alq. $4,850 ', 'venta mensual': 'Alq. $4,850 ', 'alquiler mensual': 'Alq. $4,850 ', 'likes': '6849', 'amenities': '{"amenity_[\'habitaciones\', \'wc\', \'cama\', \'plazas\', \'carro\']": ["5", "6", "6", "--", "4"]}'}


## cleaning

In [90]:
df = pd.DataFrame.from_dict(detail_features)

df.shape

(193, 7)

In [91]:
df.head(3)

,url,name,ubicacion,venta mensual,alquiler mensual,likes,amenities
0,/inmuebles/1785-casa-en-chepeconde-mar-adentro,Casa en Chepeconde Mar Adentro,Km 120 Pan. Sur 3ª fila,empty,empty,1785,"{""amenity_['habitaciones', 'wc', 'cama', 'plaz..."
1,/inmuebles/4376-casa-en-playa-chocaya,Casa en Playa Chocaya,Km 92 Pan. Sur,empty,empty,4376,"{""amenity_['habitaciones', 'wc', 'cama', 'plaz..."
2,/inmuebles/761-casa-en-bujama,Casa en Bujama,"Km 91 Pan. Sur 1ª fila Alq. $5,000","Alq. $5,000","Alq. $5,000",761,"{""amenity_['habitaciones', 'wc', 'cama', 'plaz..."


In [92]:
import re

def find_number(text):
    num = re.findall(r'[0-9]+',text)
    return " ".join(num)

def find_text(text):
    num = re.findall(r'[a-zA-Z ]+',text)
    return " ".join(num)


df['price_num']=df['alquiler mensual'].apply(lambda x: find_number(x))


df['price_text']=df['alquiler mensual'].apply(lambda x: find_text(x))

df[['uno','dos']] = df['amenities'].str.split(":",expand=True)
# https://stackoverflow.com/questions/37333299/splitting-a-pandas-dataframe-column-by-delimiter
df[['rooms','baños','camas','personas','autos']] = df['dos'].str.split(",",expand=True)

#https://stackoverflow.com/questions/13411544/delete-a-column-from-a-pandas-dataframe
df.drop(['uno', 'dos','amenities'], axis=1, inplace=True)


df['rooms']=df['rooms'].apply(lambda x: find_number(x))
df['baños']=df['baños'].apply(lambda x: find_number(x))
df['camas']=df['camas'].apply(lambda x: find_number(x))
df['personas']=df['personas'].apply(lambda x: find_number(x))
df['autos']=df['autos'].apply(lambda x: find_number(x))


df['url_casa'] = 'https://casadeplaya.com' + df['url']


In [93]:
df.head(2)

,url,name,ubicacion,venta mensual,alquiler mensual,likes,price_num,price_text,rooms,baños,camas,personas,autos,url_casa
0,/inmuebles/1785-casa-en-chepeconde-mar-adentro,Casa en Chepeconde Mar Adentro,Km 120 Pan. Sur 3ª fila,empty,empty,1785,,empty,4,3,12,,2,https://casadeplaya.com/inmuebles/1785-casa-en...
1,/inmuebles/4376-casa-en-playa-chocaya,Casa en Playa Chocaya,Km 92 Pan. Sur,empty,empty,4376,,empty,5,5,15,16,2,https://casadeplaya.com/inmuebles/4376-casa-en...


In [95]:
import openpyxl

df.to_excel(r"C:\Users\F_GODOY\Desktop\Proyectos_analytics\Scraping-BC\scraping_casas_playa2.xlsx", index = False)